In [1]:
%matplotlib inline

In [2]:
import os
import sys

In [3]:
lmu_path = os.path.abspath("../lmu")
sys.path.append(lmu_path)


In [80]:
from lmu import LMUCell

import keras as K
from keras.applications import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Input, GlobalAveragePooling2D, Conv2DTranspose, Reshape, Lambda
from keras.layers import TimeDistributed
from keras.layers.recurrent import RNN
from keras.models import Sequential, Model
from keras.initializers import Constant
from keras.utils import multi_gpu_model, to_categorical

import numpy as np

In [122]:
def lmu_layer(return_sequences=False,**kwargs):
    return RNN(LMUCell(units=6,
                       order=6,
                       theta=15,
                       input_encoders_initializer=Constant(1),
                       hidden_encoders_initializer=Constant(0),
                       memory_encoders_initializer=Constant(0),
                       input_kernel_initializer=Constant(0),
                       hidden_kernel_initializer=Constant(0),
                       memory_kernel_initializer='glorot_normal',
                      ),
               return_sequences=return_sequences,
               **kwargs)

def Lmu_stack(input_tensor, return_sequences):
    t = []
    for i in range(K.backend.int_shape(input_tensor)[-1]):
        x = Lambda(lambda x: x[...,i])(input_tensor)
        x = Reshape([input_tensor.shape[1], -1])(x)
        t.append(lmu_layer(return_sequences=return_sequences)(x))
    
    return Lambda(lambda x: K.backend.stack(x, axis=-1))(t)

    

In [132]:
seq_len = 15

# resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# for layer in resnet_model.layers:
#     layer.trainable=False
    
# base_out = resnet_model.output
# base_out = GlobalAveragePooling2D()(base_out)
# base_model = Model(inputs=resnet_model.input, outputs=base_out)

# input_layer = Input(shape=(seq_len, 224, 224, 3))

# # x = TimeDistributed(base_model)(input_layer)
# x = TimeDistributed(resnet_model)(input_layer)

i = Input(shape=(seq_len, 7, 7, 2048))

# x = Reshape(target_shape=(seq_len, 2048))(x)
out = Lmu_stack(i, return_sequences=False)

v = Reshape([-1])(out)
v = Dense(2)(v)


#Deconvolution, this is huge, how to fix?
heat = Reshape([1,1,-1])(out)
heat = Conv2DTranspose(filters=1024,kernel_size=5,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=512,kernel_size=5,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=256,kernel_size=5,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=128,kernel_size=5,strides=2,activation='relu', padding = 'same')(heat)

model = Model(inputs=i, outputs=heat)

In [133]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_135 (InputLayer)          (None, 15, 7, 7, 204 0                                            
__________________________________________________________________________________________________
lambda_246 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_247 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_248 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_249

lambda_599 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_600 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_601 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_602 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_603 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_604

lambda_734 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_735 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_736 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_737 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_738 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_739

lambda_984 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_985 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_986 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_987 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_988 (Lambda)             (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_989

__________________________________________________________________________________________________
lambda_1234 (Lambda)            (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_1235 (Lambda)            (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_1236 (Lambda)            (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_1237 (Lambda)            (None, 15, 7, 7)     0           input_135[0][0]                  
__________________________________________________________________________________________________
lambda_1238 (Lambda)            (None, 15, 7, 7)     0           input_135[0][0]                  
__________

...

rnn_5220 (RNN)                  (None, 6)            469         reshape_5297[0][0]               
__________________________________________________________________________________________________
rnn_5221 (RNN)                  (None, 6)            469         reshape_5298[0][0]               
__________________________________________________________________________________________________
rnn_5222 (RNN)                  (None, 6)            469         reshape_5299[0][0]               
__________________________________________________________________________________________________
rnn_5223 (RNN)                  (None, 6)            469         reshape_5300[0][0]               
__________________________________________________________________________________________________
rnn_5224 (RNN)                  (None, 6)            469         reshape_5301[0][0]               
__________________________________________________________________________________________________
rnn_5225 (

rnn_5508 (RNN)                  (None, 6)            469         reshape_5585[0][0]               
__________________________________________________________________________________________________
rnn_5509 (RNN)                  (None, 6)            469         reshape_5586[0][0]               
__________________________________________________________________________________________________
rnn_5510 (RNN)                  (None, 6)            469         reshape_5587[0][0]               
__________________________________________________________________________________________________
rnn_5511 (RNN)                  (None, 6)            469         reshape_5588[0][0]               
__________________________________________________________________________________________________
rnn_5512 (RNN)                  (None, 6)            469         reshape_5589[0][0]               
__________________________________________________________________________________________________
rnn_5513 (

__________________________________________________________________________________________________
rnn_6008 (RNN)                  (None, 6)            469         reshape_6085[0][0]               
__________________________________________________________________________________________________
rnn_6009 (RNN)                  (None, 6)            469         reshape_6086[0][0]               
__________________________________________________________________________________________________
rnn_6010 (RNN)                  (None, 6)            469         reshape_6087[0][0]               
__________________________________________________________________________________________________
rnn_6011 (RNN)                  (None, 6)            469         reshape_6088[0][0]               
__________________________________________________________________________________________________
rnn_6012 (RNN)                  (None, 6)            469         reshape_6089[0][0]               
__________

                                                                 rnn_4799[0][0]                   
                                                                 rnn_4800[0][0]                   
                                                                 rnn_4801[0][0]                   
                                                                 rnn_4802[0][0]                   
                                                                 rnn_4803[0][0]                   
                                                                 rnn_4804[0][0]                   
                                                                 rnn_4805[0][0]                   
                                                                 rnn_4806[0][0]                   
                                                                 rnn_4807[0][0]                   
                                                                 rnn_4808[0][0]                   
          

                                                                 rnn_5798[0][0]                   
                                                                 rnn_5799[0][0]                   
                                                                 rnn_5800[0][0]                   
                                                                 rnn_5801[0][0]                   
                                                                 rnn_5802[0][0]                   
                                                                 rnn_5803[0][0]                   
                                                                 rnn_5804[0][0]                   
                                                                 rnn_5805[0][0]                   
                                                                 rnn_5806[0][0]                   
                                                                 rnn_5807[0][0]                   
          